<a href="https://colab.research.google.com/github/Vishu254-bit/vish/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

# --- Data Loading (Placeholder) ---
# NOTE: Replace this with your actual data loading code.
# You will need to load the NSL-KDD dataset from your local files.
# Example: df = pd.read_csv('your_nsl_kdd_dataset.csv')
# For demonstration, we will create a simple mock dataset.
print("Loading dataset...")
X = np.random.rand(1000, 10) * 100 # 1000 samples, 10 features
y = np.random.randint(0, 2, 1000)   # Binary labels (0 or 1)
print("Dataset loaded successfully.")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- Define Models and Scaling Pipelines ---

def evaluate_model(model, X_train, y_train, X_test, y_test, scaler=None):
    """
    Trains and evaluates a model with an optional scaler.
    """
    if scaler:
        # Fit scaler on training data and transform both train and test data
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Train the model
        model.fit(X_train_scaled, y_train)

        # Make predictions
        y_pred = model.predict(X_test_scaled)
    else:
        # Train the model with no scaling
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    return accuracy, precision, recall, f1

# --- Run Experiments ---
svm_model = SVC(kernel='rbf', gamma='auto', random_state=42)

# 1. No Scaling (Baseline)
print("\n--- Evaluating SVM with No Scaling (Baseline) ---")
accuracy_no_scale, precision_no_scale, recall_no_scale, f1_no_scale = evaluate_model(
    svm_model, X_train, y_train, X_test, y_test
)
print(f"Accuracy: {accuracy_no_scale:.4f}")
print(f"Precision: {precision_no_scale:.4f}")
print(f"F1-Score: {f1_no_scale:.4f}")

# 2. Min-Max Normalization
print("\n--- Evaluating SVM with Min-Max Normalization ---")
minmax_scaler = MinMaxScaler()
accuracy_minmax, precision_minmax, recall_minmax, f1_minmax = evaluate_model(
    svm_model, X_train, y_train, X_test, y_test, scaler=minmax_scaler
)
print(f"Accuracy: {accuracy_minmax:.4f}")
print(f"Precision: {precision_minmax:.4f}")
print(f"F1-Score: {f1_minmax:.4f}")

# 3. Z-score Standardization
print("\n--- Evaluating SVM with Z-score Standardization ---")
zscore_scaler = StandardScaler()
accuracy_zscore, precision_zscore, recall_zscore, f1_zscore = evaluate_model(
    svm_model, X_train, y_train, X_test, y_test, scaler=zscore_scaler
)
print(f"Accuracy: {accuracy_zscore:.4f}")
print(f"Precision: {precision_zscore:.4f}")
print(f"F1-Score: {f1_zscore:.4f}")



Loading dataset...
Dataset loaded successfully.

--- Evaluating SVM with No Scaling (Baseline) ---
Accuracy: 0.5300
Precision: 0.2809
F1-Score: 0.3672

--- Evaluating SVM with Min-Max Normalization ---
Accuracy: 0.5033
Precision: 0.4991
F1-Score: 0.4987

--- Evaluating SVM with Z-score Standardization ---
Accuracy: 0.5133
Precision: 0.5099
F1-Score: 0.5098
